 # Keras Tutorial

Keras is a high-level neural networks API, written in Python and capable of running on top of certain lower-level frameworks like Tensorflow. It is one level above Tensorflow and aims at implementing the Deep Learning pipelines easily and quickly.

For our final project, we will be using Keras to build the entire pipeline from scratch. This includes data pre-processing, feature extraction (with CNN) and classification (with fully-connected nets). The goal of this tutorial is to give you adequate knowledge to prepare you for the final project.

What is good about Keras compared with other Deep Learning frameworks:
* It's high-level, which means that you can implement complex things with several lines of simple code
* It works directly with NumPy arrays, so you don't have to spend extra time on creating a Python class for dataset like PyTorch

Remember in the class we talked about the pipeline of a real computer vision system, in which we:

1. First clean the data to the format to be used for later steps (which includes data loading, data pre-processing, dataset splitting (we'll talk about this on Friday), data augmentation (which we're not gonna cover), etc);

2. Then we build the model for feature extraction as well as for final regression / classification. Remember we have many choices like linear model, fully connected neural nets, convolutional neural nets, etc. And we can implement these models very easily in Keras with just one line of code;

3. After we get the data and the model, we need to code up the optimization part (for which we'll use gradient descent). 

In this tutorial, we'll go over these parts sequentially.

## Data Loading and Pre-processing

So in Keras we don't need anything specific for data, we just use NumPy and represent our data in Numpy arrays. Now we're gonna create some fake data to be used later.

In [1]:
# Import necessary packages
import numpy as np

In [2]:
# Create random numpy arrays (ldata loading)
rand_data = np.random.random((1000, 32, 32, 3)) # We have 1000 fake images with spatial size 32 * 32
rand_label = np.array([0]*500 + [1]*500)        # Create fake binary labels for these images  

print(rand_data.shape)

(1000, 32, 32, 3)


In [3]:
# Split data into train, validation and test sets (we'll talk more about this on Friday)
train_ratio, val_ratio = 0.9, 0.05

X_train = rand_data[:int(rand_data.shape[0]*train_ratio), ...] # ... means all the other axes
y_train = rand_label[:int(rand_data.shape[0]*train_ratio), ...]

X_val = rand_data[int(rand_data.shape[0]*train_ratio):int(rand_data.shape[0]*(train_ratio+val_ratio)), ...]
y_val = rand_label[int(rand_data.shape[0]*train_ratio):int(rand_data.shape[0]*(train_ratio+val_ratio)), ...]

X_test = rand_data[int(rand_data.shape[0]*(train_ratio+val_ratio)):, ...]
y_test = rand_label[int(rand_data.shape[0]*(train_ratio+val_ratio)):, ...]

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(900, 32, 32, 3)
(50, 32, 32, 3)
(50, 32, 32, 3)


## Model construction

Now we have all the data, next we're gonna build our model for feature extraction as well as classification. In Keras, you can easily build many models, as shown below.

In [4]:
import keras
from keras.models import Sequential # Sequential is one of the main models in Keras, which is basically a sequentially stacked series of layers

model = Sequential() # Initialize a Sequential model instance

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
# First we'll use fully-connected neural nets
from keras.layers import Dense # Dense is Keras's name for fully connected layers

# We can stack layers like lego blocks by simplying using `add()`
# `units` is the number of neurons
# `activation` is the nonlinear function we add for each layer
# We only need to specify `input_dim` which is the input dimension for the layer for the input layer, because for later layers the input is just the output from last layer
# Once again, the number of neurons in hidden layers (e.g., 64 and 16 here) are design choices

model.add(Dense(units=64, activation='sigmoid', input_dim=32*32*3)) 
model.add(Dense(units=16, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

In [6]:
# Once the model is build, we then configure the learning process with `compile()`
# We need to specify the loss function, the optimizer and the metric we use to evaluate our model
# For loss here we're using a function called binary cross-entropy loss, which is specifically for binary classification
# For optimizer we're using gradient descent, which is written as 'sgd' in Keras
# Since we're doing classification, normally the classification accuracy is how we evaluate the model

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [7]:
# The above is actually a convenient way that Keras provides for easy implementation. If you want to have more control over the learning process (e.g., the learning rate), you can use the following:

model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.SGD(lr=0.001))

In [8]:
# Up to this point we're all doing configurations. Now everything is set up so we're letting the model do real things!

# Since now we're using a fully-connected nets, remember we need to flatten the image to a single long vector first
X_train_flat = X_train.reshape((-1, 32*32*3)) # -1 means letting NumPy to figure this axis out automatically
X_val_flat = X_val.reshape((-1, 32*32*3))
X_test_flat = X_test.reshape((-1, 32*32*3))

print(X_train_flat.shape)
print(X_val_flat.shape)
print(X_test_flat.shape)

# Then use fit() to actually train our model
# epochs is basically how many iterations we want for the update process. The model needs some time to reach the optimal state!
# batch_size is how many images we use each time to estimate the gradient. Remember that the more we use the more accurate each update will be, but it will also be slower

model.fit(X_train_flat, y_train, epochs=5, batch_size=32, validation_data=(X_val_flat, y_val))

(900, 3072)
(50, 3072)
(50, 3072)
Train on 900 samples, validate on 50 samples
Epoch 1/5
900/900 [==============================] - 0s 469us/step - loss: 0.7009 - val_loss: 0.6465
Epoch 2/5
900/900 [==============================] - 0s 104us/step - loss: 0.6989 - val_loss: 0.6585
Epoch 3/5
900/900 [==============================] - 0s 87us/step - loss: 0.6971 - val_loss: 0.6710
Epoch 4/5
900/900 [==============================] - 0s 104us/step - loss: 0.6954 - val_loss: 0.6826
Epoch 5/5
900/900 [==============================] - 0s 104us/step - loss: 0.6940 - val_loss: 0.6945


In [9]:
# Now let's see how our model does
acc = model.evaluate(X_test_flat, y_test)
print('The test accuracy is: {}'.format(acc))

# And make predictions
prob = model.predict(X_test_flat) # These are probabilities, and we want to convert them to class labels
label = np.array(prob > 0.5, dtype=int)

print('The predicted probabilities are: {}'.format(prob))
print('The predicted class labels are: {}'.format(label))


50/50 [==============================] - 0s 625us/step
The test accuracy is: 0.7019074392318726
The predicted probabilities are: [[0.48876253]
 [0.51817054]
 [0.5061766 ]
 [0.494383  ]
 [0.49292699]
 [0.49779132]
 [0.49732846]
 [0.48806584]
 [0.48646295]
 [0.5188182 ]
 [0.50859994]
 [0.49428093]
 [0.4848431 ]
 [0.49128535]
 [0.49408555]
 [0.4938376 ]
 [0.4905901 ]
 [0.49060732]
 [0.48320913]
 [0.48246092]
 [0.4910382 ]
 [0.4972248 ]
 [0.48123184]
 [0.50067025]
 [0.50971246]
 [0.48304936]
 [0.5047053 ]
 [0.49468285]
 [0.4956223 ]
 [0.49726537]
 [0.5106281 ]
 [0.50213635]
 [0.50250816]
 [0.48997217]
 [0.4951935 ]
 [0.48122647]
 [0.49782285]
 [0.49421945]
 [0.503532  ]
 [0.4958118 ]
 [0.49407437]
 [0.49981916]
 [0.5083103 ]
 [0.49968585]
 [0.49011898]
 [0.49148956]
 [0.4916731 ]
 [0.4954145 ]
 [0.49237582]
 [0.491713  ]]
The predicted class labels are: [[0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0

In [10]:
# As we can expect, the results are totally random
# You can also play with other models, e.g., convnets
# So we do the same procedure once more

model = Sequential() # Re-initialize the model

# Feature extractor
# We're using such an architecture: conv -> maxpool -> conv -> maxpool
# 'same' padding means we zero-pad the images so that the output will be of the same size as the input
model.add(keras.layers.Conv2D(filters=16, kernel_size=3, strides=(2, 2), padding='same'))
model.add(keras.layers.Activation('sigmoid'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2))) # By default the stride is the same as the pooling size

model.add(keras.layers.Conv2D(filters=32, kernel_size=2, strides=(1, 1), padding='same'))
model.add(keras.layers.Activation('relu')) # ReLU is another kind of non-linear function
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Classifier
# We're using a 2-layer FC net for classification 
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(32))
model.add(keras.layers.Activation('relu'))

model.add(keras.layers.Dense(1))
model.add(keras.layers.Activation('sigmoid'))

# Compilation
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.SGD(lr=0.001))

# Training
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluation
acc = model.evaluate(X_test, y_test)
print('The test accuracy is: {}'.format(acc))

# And make predictions
prob = model.predict(X_test) # These are probabilities, and we want to convert them to class labels
label = np.array(prob > 0.5, dtype=int)

print('The predicted probabilities are: {}'.format(prob))
print('The predicted class labels are: {}'.format(label))

Train on 900 samples, validate on 50 samples
Epoch 1/10
900/900 [==============================] - 0s 538us/step - loss: 0.6915 - val_loss: 0.8827
Epoch 2/10
900/900 [==============================] - 0s 191us/step - loss: 0.6887 - val_loss: 0.8492
Epoch 3/10
900/900 [==============================] - 0s 191us/step - loss: 0.6877 - val_loss: 0.8423
Epoch 4/10
900/900 [==============================] - 0s 243us/step - loss: 0.6876 - val_loss: 0.8346
Epoch 5/10
900/900 [==============================] - 0s 174us/step - loss: 0.6876 - val_loss: 0.8264
Epoch 6/10
900/900 [==============================] - 0s 208us/step - loss: 0.6875 - val_loss: 0.8344
Epoch 7/10
900/900 [==============================] - 0s 208us/step - loss: 0.6875 - val_loss: 0.8283
Epoch 8/10
900/900 [==============================] - 0s 174us/step - loss: 0.6874 - val_loss: 0.8133
Epoch 9/10
900/900 [==============================] - 0s 191us/step - loss: 0.6875 - val_loss: 0.8093
Epoch 10/10
50/50 [==================